## test parcellation approach with subj05

In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import h5py
import numpy as np
import scipy as sp
import scipy.stats as stats
import nibabel as nibabel
import pandas as pd
import nibabel.freesurfer.mghformat as mgh
import scipy.io
import itertools 
import pickle

In [6]:
import sys
utils_dir = '/oak/stanford/groups/kalanit/biac2/kgs/projects/Dawn/NSD/code/streams/utils/'
sys.path.append(utils_dir)

In [7]:
from rsm_utils import get_flat_lower_tri, make_flat_rsms

In [8]:
data_dir = '../../../data/'
local_data_dir = '../../../local_data/'

subjid = ['06']
n_repeats = 3

#threshold for voxels (based on split-half reliability)
thresh = 0.1

In [9]:
#get ROI data
rh_parcels = []
for sidx, sid in enumerate(subjid):
    mgh_file = mgh.load(local_data_dir+'freesurfer/subj'+ sid +'/lh.tessellate_300.mgz')
    rh_parcels.append(mgh_file.get_fdata()[:,0,0])

In [10]:
num_rois = int(np.max(rh_parcels))

In [ ]:
#get voxel level split-half reliability data
reliability = []
for sidx, sid in enumerate(subjid):
    
    sh_dir = local_data_dir + 'freesurfer/subj' + sid + '/lh_split_half.mat'
    sh = scipy.io.loadmat(sh_dir)
    
    reliability.append(sh['mean'])

In [ ]:
# let's organize by ROI 
sh_by_ROI = [[[] for j in range(num_rois)] for i in range(len(subjid))]
total_vox = np.zeros((len(subjid), num_rois))

for sidx, sid in enumerate(subjid):  
    for roi_idx in range(num_rois):       
        sh_by_ROI[sidx][roi_idx]=reliability[sidx][:,rh_parcels[sidx] == roi_idx+1]
        total_vox[sidx,roi_idx] = len(sh_by_ROI[sidx][roi_idx][0])

In [ ]:
thresh_rois = np.array([np.sum(sh_by_ROI[0][r]>.1) for r in range(num_rois)])

In [ ]:
thresh_rois

In [ ]:
np.sum(thresh_rois>0)/num_rois #percent rois above thres

In [11]:
all_ids = []
max_session = np.zeros(len(subjid))
for sidx, sid in enumerate(subjid):
    
    data = pd.read_csv(data_dir+'nsddata/ppdata/subj'+ sid +'/behav/responses.tsv', sep='\t')
    
    max_session[sidx] = np.max(np.array(data['SESSION'])) 
    
    all_ids.append(np.array(data['73KID']))
    
    #shared_mask.append(np.isin(all_ids[sidx],sharedix))

In [12]:
which_reps = []
for sidx, sid in enumerate(subjid):
    vals, idx_start, count = np.unique(all_ids[sidx], return_counts=True,
                                    return_index=True)
    which_reps.append(vals[count == n_repeats])
    
least_trials = min(which_reps, key=len)

In [13]:
id_nums_3reps = []
mask_3reps = []
for sidx, sid in enumerate(subjid):
    
    data = pd.read_csv(data_dir+'nsddata/ppdata/subj'+ sid +'/behav/responses.tsv', sep='\t')
    
    mask_3reps.append(np.isin(all_ids[sidx],which_reps[sidx]))
    id_nums_3reps.append(np.array(data['73KID'])[mask_3reps[sidx]])


In [14]:
arr1inds = id_nums_3reps[sidx].argsort()

In [15]:
#right hemisphere
betas_by_ROI = [[] for j in range(num_rois)]

for sidx, sid in enumerate(subjid):
    
    print(sidx)
    mask = mask_3reps[sidx]
    sorted_betas = []
    
    #get all betas across all sessions
    for sess in range(1,int(max_session[sidx])+1):
        print(sess)
                
        if(sess < 10):
            idx = '0' + str(sess)
        else:
            idx = str(sess)

        raw_betas = h5py.File(local_data_dir+'freesurfer/subj'+sid+'/betas/lh.zscore_betas_session'+idx+'.hdf5','r')

        sess_betas = raw_betas['zscore_betas'][:][mask[(sess-1)*750:sess*750]]
        del raw_betas
        
        if(sess==1):
            for roi_idx in range(num_rois):
                betas_by_ROI[roi_idx] = sess_betas[:,rh_parcels[sidx] == roi_idx+1]
        else:
            for roi_idx in range(num_rois):
                betas_by_ROI[roi_idx] = np.append(betas_by_ROI[roi_idx],sess_betas[:,rh_parcels[sidx] == roi_idx+1],axis=0)
        
        del sess_betas


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [16]:
betas_by_repeat_by_ROI = [[[] for j in range(num_rois)] for i in range(len(subjid))]
for sidx, sid in enumerate(subjid):
    for roi_idx in range(num_rois):  
        
        sorted_betas = betas_by_ROI[roi_idx][arr1inds[::-1]]
        
        for r in range(n_repeats):
            betas_by_repeat_by_ROI[sidx][roi_idx].insert(r,sorted_betas[r::3])

In [17]:
del betas_by_ROI

In [14]:
betas_by_repeat_by_ROI[0][0][0].shape

(6234, 626)

In [ ]:
#Replace voxels with split-half reliability < thresh with NaNs and then trim those from data structure

#convert to nans
for sidx, sid in enumerate(subjid):  
    for roi_idx in range(num_rois): 
        for vox in range(len(sh_by_ROI[sidx][roi_idx][0])):
            if sh_by_ROI[sidx][roi_idx][0][vox] < thresh:
                betas_by_repeat_by_ROI[sidx][roi_idx][0][:,vox]=np.nan
                betas_by_repeat_by_ROI[sidx][roi_idx][1][:,vox]=np.nan
                betas_by_repeat_by_ROI[sidx][roi_idx][2][:,vox]=np.nan    
#trim out nans
for sidx, sid in enumerate(subjid):   
    for roi_idx in range(num_rois): 
        for r in range(n_repeats):
            temp = betas_by_repeat_by_ROI[sidx][roi_idx][r]
            trimmed = temp[:,~np.all(np.isnan(temp), axis=0)]

            betas_by_repeat_by_ROI[sidx][roi_idx][r] = trimmed

In [ ]:
del sorted_betas

In [ ]:
del betas_by_ROI

In [ ]:
del trimmed

In [ ]:
test.shape

In [15]:
def np_pearson_cor(x, y):
    xv = x - x.mean(axis=0)
    yv = y - y.mean(axis=0)
    xvss = (xv * xv).sum(axis=0)
    yvss = (yv * yv).sum(axis=0)
    result = np.matmul(xv.transpose(), yv) / np.sqrt(np.outer(xvss, yvss))
    # bound the values to -1 to 1 in the event of precision issues
    return np.maximum(np.minimum(result, 1.0), -1.0)

In [ ]:
def ss(a, axis):
    return np.sum(a * a, axis=axis)

def compute_corr(x, y):
    x = np.asarray(x)
    y = np.asarray(y)
    mx = x.mean(axis=-1)
    my = y.mean(axis=-1)
    xm, ym = x - mx[..., None], y - my[..., None]
    r_num = np.add.reduce(xm * ym, axis=-1)
    r_den = np.sqrt(ss(xm, axis=-1) * ss(ym, axis=-1))
    r = r_num / r_den
    return r

In [ ]:
testing = np_pearson_cor(test.T,test.T)

In [ ]:
testing2 = compute_corr(test,test)

In [ ]:
testing2.shape

In [ ]:
round_test = np.around(rsm, decimals = 5)
round_testing = np.around(testing, decimals = 5)

In [ ]:
round_test == round_testing

In [ ]:
testing == rsm

In [ ]:
roi_idx = 0
r = 0
test = betas_by_repeat_by_ROI[0][roi_idx][r].T

In [ ]:
timeit rsm = np.corrcoef(test)

In [ ]:
timeit testing = np_pearson_cor(test,test)

In [ ]:
rsm = np.corrcoef(test)
rsm.shape

In [16]:
tril_flat_shape = int((betas_by_repeat_by_ROI[0][0][0].shape[0]**2/2) - (betas_by_repeat_by_ROI[0][0][0].shape[0]/2))
flat_rsm = np.zeros((num_rois, tril_flat_shape, n_repeats), dtype=int)


In [17]:
flat_rsm.shape

(68, 19428261, 3)

In [112]:
roi_idx = 0
r = 0
flip = betas_by_repeat_by_ROI[sidx][roi_idx][r].T
rsm = np_pearson_cor(flip,flip)

In [113]:
rsm_old = np.corrcoef(betas_by_repeat_by_ROI[sidx][roi_idx][r])

In [57]:
rsm_old

array([[ 1.        ,  0.00504949,  0.23769638, ...,  0.13514723,
        -0.11486686, -0.26558006],
       [ 0.00504949,  1.        ,  0.01949109, ...,  0.26568577,
         0.2418056 ,  0.18465994],
       [ 0.23769638,  0.01949109,  1.        , ...,  0.48112218,
        -0.25348294, -0.25367927],
       ...,
       [ 0.13514723,  0.26568577,  0.48112218, ...,  1.        ,
        -0.0570444 , -0.29417565],
       [-0.11486686,  0.2418056 , -0.25348294, ..., -0.0570444 ,
         1.        ,  0.11984408],
       [-0.26558006,  0.18465994, -0.25367927, ..., -0.29417565,
         0.11984408,  1.        ]])

In [58]:
rsm

array([[ 1.        ,  0.00504949,  0.23769638, ...,  0.13514723,
        -0.11486686, -0.26558006],
       [ 0.00504949,  1.        ,  0.01949109, ...,  0.26568577,
         0.2418056 ,  0.18465994],
       [ 0.23769638,  0.01949109,  1.        , ...,  0.48112218,
        -0.25348294, -0.25367927],
       ...,
       [ 0.13514723,  0.26568577,  0.48112218, ...,  1.        ,
        -0.0570444 , -0.29417565],
       [-0.11486686,  0.2418056 , -0.25348294, ..., -0.0570444 ,
         1.        ,  0.11984408],
       [-0.26558006,  0.18465994, -0.25367927, ..., -0.29417565,
         0.11984408,  1.        ]])

In [26]:
get_flat_lower_tri(test,diagonal=False).shape

(19424752,)

In [22]:
get_flat_lower_tri(rsm,diagonal=False).shape

(19428261,)

In [70]:
lower_tri_test = np.tril(test, -1).T.ravel()
lower_tri_pre = np.tril(rsm, -1).T.ravel()

In [85]:
(lower_tri_test==0).shape

(38862756,)

In [84]:
(lower_tri_pre==0).shape

(38862756,)

In [28]:
test = np.around(rsm*10000,0)
#test[test == 10000] = 1 #set diagonal back to 1

In [30]:
lower_tri = np.tril(test, -1).T.ravel()

In [86]:
lower_tri_pre[lower_tri_pre != 0].shape

(19428261,)

In [87]:
lower_tri_test[lower_tri_test != 0].shape

(19424752,)

In [41]:
lower_tri = np.tril(rsm, -1)

In [109]:
rsm[idx].shape

(19434495,)

In [117]:
idx = np.tril_indices(rsm.shape[0], -1)

In [120]:
rsm[idx].shape

(19428261,)

In [119]:
get_flat_lower_tri(rsm_old,diagonal=False).shape

(19428261,)

In [121]:
#Create RSMS for all the ROIs, repeats and subjects
tril_flat_shape = int((betas_by_repeat_by_ROI[0][0][0].shape[0]**2/2) - (betas_by_repeat_by_ROI[0][0][0].shape[0]/2))
flat_rsm = np.zeros((num_rois, tril_flat_shape, n_repeats), dtype=int)

sidx = 0 #currently doing one subject at a time
for roi_idx in range(num_rois):
    for r in range(n_repeats):
        flip = betas_by_repeat_by_ROI[sidx][roi_idx][r].T
        rsm = np_pearson_cor(flip,flip)
        test = np.around(rsm*1000,0)
        idx = np.tril_indices(test.shape[0], -1)
        flat_rsm[roi_idx, :, r] = test[idx]

In [ ]:
get_flat_lower_tri(test,diagonal=False).shape

In [ ]:
betas_by_repeat_by_ROI[0][0][0].shape[0]**2/2 - (betas_by_repeat_by_ROI[0][0][0].shape[0])

In [ ]:
timeit stats.pearsonr(flat_rsm[0,:,0],flat_rsm[1,:,0])[0]

In [ ]:
stats.pearsonr(flat_rsm[0,:,0],flat_rsm[1,:,0])[0]

In [ ]:
timeit np_pearson_cor(flat_rsm[0,:,0],flat_rsm[1,:,0])[0][0]

In [ ]:
np_pearson_cor(flat_rsm[0,:,0],flat_rsm[1,:,0])[0][0]

In [ ]:
timeit compute_corr(flat_rsm[0,:,0],flat_rsm[1,:,0])

In [ ]:
#Create RSMS for all the ROIs, repeats and subjects
tril_flat_shape = int((betas_by_repeat_by_ROI[0][0][0].shape[0]**2/2) - (betas_by_repeat_by_ROI[0][0][0].shape[0]/2))
flat_rsm_r1 = np.zeros((num_rois, tril_flat_shape))
flat_rsm_r2 = np.zeros((num_rois, tril_flat_shape))
flat_rsm_r3 = np.zeros((num_rois, tril_flat_shape))


In [ ]:
sidx = 0
for roi_idx in range(num_rois):
        
    rsm = np.corrcoef(betas_by_repeat_by_ROI[sidx][roi_idx][0])
    flat_rsm_r1[roi_idx, :] = get_flat_lower_tri(rsm,diagonal=False)

    rsm = np.corrcoef(betas_by_repeat_by_ROI[sidx][roi_idx][1])
    flat_rsm_r2[roi_idx, :] = get_flat_lower_tri(rsm,diagonal=False)

    rsm = np.corrcoef(betas_by_repeat_by_ROI[sidx][roi_idx][2])
    flat_rsm_r3[roi_idx, :] = get_flat_lower_tri(rsm,diagonal=False)


In [ ]:
flat_rsm_r1

In [ ]:
r1_trial_order = [0, 0, 1, 1, 2, 2]
r2_trial_order = [1, 2, 0, 2, 0, 1]

In [ ]:
del betas_by_repeat_by_ROI

In [ ]:
del rsm

In [ ]:
mega_matrix = np.zeros((num_rois,num_rois))

for roi_idx1 in range(num_rois): #rows - i.e. model candidate
    
    split_half = np.zeros((3))
    split_half = [stats.pearsonr(flat_rsm_r1[roi_idx1,:],flat_rsm_r2[roi_idx1,:])[0],
                  stats.pearsonr(flat_rsm_r1[roi_idx1,:],flat_rsm_r3[roi_idx1,:])[0],
                  stats.pearsonr(flat_rsm_r2[roi_idx1,:],flat_rsm_r3[roi_idx1,:])[0]]
    NC_model = np.mean(split_half) * 100
    
    for roi_idx2 in range(num_rois): #columns - i.e. target data
        
        split_half = np.zeros((3))
        split_half = [stats.pearsonr(flat_rsm_r1[roi_idx2,:],flat_rsm_r2[roi_idx2,:])[0],
                      stats.pearsonr(flat_rsm_r1[roi_idx2,:],flat_rsm_r3[roi_idx2,:])[0],
                      stats.pearsonr(flat_rsm_r2[roi_idx2,:],flat_rsm_r3[roi_idx2,:])[0]]
        NC_target = np.mean(split_half) * 100
        
        
        rsm_corr = np.zeros((6))
        
        rsm_corr[0] = stats.pearsonr(flat_rsm_r1[roi_idx1,:],
                                     flat_rsm_r2[roi_idx2,:])[0]
        rsm_corr[1] = stats.pearsonr(flat_rsm_r1[roi_idx1,:],
                                     flat_rsm_r3[roi_idx2,:])[0]
        rsm_corr[2] = stats.pearsonr(flat_rsm_r2[roi_idx1,:],
                                     flat_rsm_r1[roi_idx2,:])[0]
        rsm_corr[3] = stats.pearsonr(flat_rsm_r2[roi_idx1,:],
                                     flat_rsm_r3[roi_idx2,:])[0]
        rsm_corr[4] = stats.pearsonr(flat_rsm_r3[roi_idx1,:],
                                     flat_rsm_r1[roi_idx2,:])[0]
        rsm_corr[5] = stats.pearsonr(flat_rsm_r3[roi_idx1,:],
                                     flat_rsm_r2[roi_idx2,:])[0]
        
        mega_matrix[roi_idx1,roi_idx2] = np.mean(rsm_corr) * np.sqrt(100/NC_model) * np.sqrt(100/NC_target)
        

In [ ]:
plt.imshow(mega_matrix)
plt.clim(0,1)
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
from sklearn.manifold import MDS
x, y = MDS(dissimilarity='precomputed').fit_transform(1-mega_matrix).T

In [ ]:
plt.scatter(x, y)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, set_link_color_palette
from scipy.spatial.distance import squareform

In [ ]:
sym_mm = np.around(mega_matrix, decimals = 5)

In [ ]:
dists = squareform(1-sym_mm)

In [ ]:
linkage_matrix = linkage(dists, "complete", optimal_ordering=True)


In [ ]:
dn = dendrogram(linkage_matrix,
                above_threshold_color='#bcbddc')